In [40]:
offset = 0
limit = 300
period = 'max' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

In [41]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = data_clean['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 11508


In [42]:
! pip install yfinance > /dev/null 2>&1
! mkdir hist

mkdir: hist: File exists


In [43]:
import yfinance as yf
import os, contextlib

In [44]:
%%time

limit = limit if limit else len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            data.to_csv('hist/{}.csv'.format(s))

print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))

Total number of valid symbols downloaded = 262
CPU times: user 16 s, sys: 768 ms, total: 16.8 s
Wall time: 1min 16s


In [45]:
valid_data = data_clean[is_valid]
valid_data.to_csv('symbols_valid_meta.csv', index=False)

In [46]:
valid_data

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
0,Y,A,"Agilent Technologies, Inc. Common Stock",N,,N,100.0,N,NaN,A,A,N
1,Y,AA,Alcoa Corporation Common Stock,N,,N,100.0,N,NaN,AA,AA,N
2,Y,AAA,Investment Managers Series Trust II AXS First ...,P,,Y,100.0,N,NaN,AAA,AAA,N
3,Y,AAAU,Goldman Sachs Physical Gold ETF Shares,Z,,Y,100.0,N,NaN,AAAU,AAAU,N
4,Y,AAC,Ares Acquisition Corporation Class A Ordinary ...,N,,N,100.0,N,NaN,AAC,AAC,N
...,...,...,...,...,...,...,...,...,...,...,...,...
295,Y,AGTI,"Agiliti, Inc. Common Stock",N,,N,100.0,N,NaN,AGTI,AGTI,N
296,Y,AGX,"Argan, Inc. Common Stock",N,,N,100.0,N,NaN,AGX,AGX,N
297,Y,AGYS,"Agilysys, Inc. - Common Stock",Q,Q,N,100.0,N,N,NaN,AGYS,N
298,Y,AGZ,iShares Agency Bond ETF,P,,Y,100.0,N,NaN,AGZ,AGZ,N


In [47]:
!mkdir stocks
!mkdir etfs

mkdir: stocks: File exists
mkdir: etfs: File exists


In [48]:
etfs = valid_data[valid_data['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = valid_data[valid_data['ETF'] == 'N']['NASDAQ Symbol'].tolist()

In [49]:
len(etfs)

39

In [50]:
len(stocks)

223

In [51]:
import shutil
from os.path import isfile, join

def move_symbols(symbols, dest):
    for s in symbols:
        filename = '{}.csv'.format(s)
        shutil.move(join('hist', filename), join(dest, filename))
        
move_symbols(etfs, "etfs")
move_symbols(stocks, "stocks")

In [52]:
! rmdir hist

## Choose some of the stocks

In [53]:
# apple = pd.read_csv("~/Documents/Tableau/Nasdaq/stocks/AAPL.csv") - this one is already dated
# I'll get the latest from yahoo finance today along with the rest
apple = yf.download('AAPL')
apple.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-04-10,161.419998,162.029999,160.080002,162.029999,162.029999,47716900
2023-04-11,162.350006,162.360001,160.509995,160.800003,160.800003,47644200
2023-04-12,161.220001,162.059998,159.779999,160.100006,160.100006,50133100
2023-04-13,161.630005,165.800003,161.419998,165.559998,165.559998,68445600
2023-04-14,164.589996,166.320007,163.820007,165.210007,165.210007,49337200


For this project, just get a few more specific stocks.

In [54]:
google = yf.download('GOOGL')

[*********************100%***********************]  1 of 1 completed


In [55]:
facebook = yf.download('META')
facebook.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-05-18,42.049999,45.000000,38.000000,38.230000,38.230000,573576400
2012-05-21,36.529999,36.660000,33.000000,34.029999,34.029999,168192700
2012-05-22,32.610001,33.590000,30.940001,31.000000,31.000000,101786600
2012-05-23,31.370001,32.500000,31.360001,32.000000,32.000000,73600000
2012-05-24,32.950001,33.209999,31.770000,33.029999,33.029999,50237200


In [56]:
facebook.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-04-10,214.710007,215.660004,210.660004,214.750000,214.750000,16106100
2023-04-11,215.479996,216.020004,213.410004,213.850006,213.850006,16710100
2023-04-12,214.839996,216.839996,212.580002,214.000000,214.000000,18972200
2023-04-13,215.729996,221.149994,215.690002,220.350006,220.350006,23310400
2023-04-14,217.880005,222.110001,217.550003,221.490005,221.490005,21574200


In [57]:
nvidia = yf.download('NVDA')
tesla = yf.download('TSLA')
#  twitter = yf.download('TWTR') - no longer publicly traded

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [58]:
microsoft = yf.download('MSFT')

[*********************100%***********************]  1 of 1 completed


In [59]:
dfs = [apple, facebook, google, microsoft, nvidia, tesla]

In [60]:
# Make columns for moving (rolling) averages in each dataframe
for df in dfs:
    df['MA50'] = df.Close.rolling(50).mean()
    df['MA200'] = df.Close.rolling(200).mean()

In [61]:
apple.iloc[198:205,]

,Open,High,Low,Close,Adj Close,Volume,MA50,MA200
Date,,,,,,,,
1981-09-25,0.064732,0.064732,0.063616,0.063616,0.049427,34608000,0.095714,NaN
1981-09-28,0.064174,0.064732,0.064174,0.064174,0.049861,91728000,0.094687,0.118125
1981-09-29,0.067522,0.068080,0.067522,0.067522,0.052462,94684800,0.093884,0.117821
1981-09-30,0.068080,0.068638,0.068080,0.068080,0.052896,49996800,0.093103,0.117553
1981-10-01,0.068080,0.068638,0.068080,0.068080,0.052896,61129600,0.092444,0.117330
1981-10-02,0.073661,0.074219,0.073661,0.073661,0.057232,45046400,0.091841,0.117120
1981-10-05,0.075893,0.077009,0.075893,0.075893,0.058966,43097600,0.091216,0.116906


In [62]:
for df in dfs:
    df['PreviousDayClose'] = df.Close.shift(1)
apple.tail()

,Open,High,Low,Close,Adj Close,Volume,MA50,MA200,PreviousDayClose
Date,,,,,,,,,
2023-04-10,161.419998,162.029999,160.080002,162.029999,162.029999,47716900,153.761199,149.26815,164.660004
2023-04-11,162.350006,162.360001,160.509995,160.800003,160.800003,47644200,154.058600,149.38080,162.029999
2023-04-12,161.220001,162.059998,159.779999,160.100006,160.100006,50133100,154.400600,149.47300,160.800003
2023-04-13,161.630005,165.800003,161.419998,165.559998,165.559998,68445600,154.826000,149.59250,160.100006
2023-04-14,164.589996,166.320007,163.820007,165.210007,165.210007,49337200,155.221600,149.73135,165.559998


In [63]:
apple.head()

,Open,High,Low,Close,Adj Close,Volume,MA50,MA200,PreviousDayClose
Date,,,,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099722,469033600,NaN,NaN,NaN
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094519,175884800,NaN,NaN,0.128348
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087582,105728000,NaN,NaN,0.121652
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089749,86441600,NaN,NaN,0.112723
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092351,73449600,NaN,NaN,0.115513


In [64]:
for df in dfs:
    df['PercentChangeClose'] = df.Close.pct_change()

In [65]:
apple.head()

,Open,High,Low,Close,Adj Close,Volume,MA50,MA200,PreviousDayClose,PercentChangeClose
Date,,,,,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099722,469033600,NaN,NaN,NaN,NaN
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094519,175884800,NaN,NaN,0.128348,-0.052171
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087582,105728000,NaN,NaN,0.121652,-0.073398
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089749,86441600,NaN,NaN,0.112723,0.024751
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092351,73449600,NaN,NaN,0.115513,0.028992


In [66]:
for df in dfs:
    df['PreviousDayVolume'] = df.Volume.shift(1)
    df['PercentChangeVolume'] = df.Volume.pct_change()
apple.head()

,Open,High,Low,Close,Adj Close,Volume,MA50,MA200,PreviousDayClose,PercentChangeClose,PreviousDayVolume,PercentChangeVolume
Date,,,,,,,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099722,469033600,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094519,175884800,NaN,NaN,0.128348,-0.052171,469033600.0,-0.625006
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087582,105728000,NaN,NaN,0.121652,-0.073398,175884800.0,-0.398879
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089749,86441600,NaN,NaN,0.112723,0.024751,105728000.0,-0.182415
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092351,73449600,NaN,NaN,0.115513,0.028992,86441600.0,-0.150298


In [67]:
apple.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10674 entries, 1980-12-12 to 2023-04-14
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Open                 10674 non-null  float64
 1   High                 10674 non-null  float64
 2   Low                  10674 non-null  float64
 3   Close                10674 non-null  float64
 4   Adj Close            10674 non-null  float64
 5   Volume               10674 non-null  int64  
 6   MA50                 10625 non-null  float64
 7   MA200                10475 non-null  float64
 8   PreviousDayClose     10673 non-null  float64
 9   PercentChangeClose   10673 non-null  float64
 10  PreviousDayVolume    10673 non-null  float64
 11  PercentChangeVolume  10673 non-null  float64
dtypes: float64(11), int64(1)
memory usage: 1.1 MB


In [68]:
# save all as csv's to use in Tableau dashboard
apple.to_csv('apple.csv')
facebook.to_csv('facebook.csv')
google.to_csv('google.csv')
microsoft.to_csv('microsoft.csv')
nvidia.to_csv('nvidia.csv')
tesla.to_csv('tesla.csv')